In [1]:
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense
import seaborn as sns
import pandas as pd
from math import e,sqrt
from scipy.optimize import least_squares
from scipy.optimize import leastsq

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import datetime
import statistics
import math
import os
import sys
import statistics
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")

from embfts.util.DataSetUtil import DataSetUtil
from embfts.util.StatisticsUtil import StatisticsUtil

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor


In [2]:
data_set_util = DataSetUtil()
statistics_util = StatisticsUtil()

In [3]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [4]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/AirQualityUCI.csv', sep=';', decimal=',')
df = df.drop(labels=['Date','Time','Unnamed: 15', 'Unnamed: 16'], axis=1)
df.dropna(inplace=True)
#data = clean_dataset(data)
data = data_set_util.series_to_supervised_mimo(df, 1, 1)
data.head()


,CO(GT)(t-1),PT08.S1(CO)(t-1),NMHC(GT)(t-1),C6H6(GT)(t-1),PT08.S2(NMHC)(t-1),NOx(GT)(t-1),PT08.S3(NOx)(t-1),NO2(GT)(t-1),PT08.S4(NO2)(t-1),PT08.S5(O3)(t-1),...,C6H6(GT)(t),PT08.S2(NMHC)(t),NOx(GT)(t),PT08.S3(NOx)(t),NO2(GT)(t),PT08.S4(NO2)(t),PT08.S5(O3)(t),T(t),RH(t),AH(t)
1,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,...,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,...,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,...,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,...,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888
5,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,...,4.7,750.0,89.0,1337.0,96.0,1393.0,949.0,11.2,59.2,0.7848


In [5]:
# from sklearn.model_selection import train_test_split
# x,y=data.loc[:,:],data.loc[:,:]
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# # Get the number of inputs and outputs from the dataset
# df_trainx=x_train.loc[:,'CO(GT)(t-1)':'AH(t-1)']
# df_testx=x_test.loc[:,'CO(GT)(t-1)':'AH(t-1)']
# df_trainy=y_train.loc[:,'CO(GT)(t)':'AH(t)']
# df_original=y_test.loc[:,'CO(GT)(t)':'AH(t)']

# n_inputs, n_outputs = x.shape[1], y.shape[1]

In [6]:
# MultiOutputRegressor(estimator=RandomForestRegressor(criterion='friedman_mse',
#                                                      max_depth=21,
#                                                      max_leaf_nodes=20,
#                                                      min_samples_leaf=2,
#                                                      n_estimators=300))


def create_and_fit_model(X_train,y_train):
    clf = MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=20,
                                                     max_leaf_nodes=50,
                                                     min_samples_leaf=6,
                                                     n_estimators=50))
    clf.fit(X_train, y_train)
    
    return clf

In [7]:
def forecast_model(clf, X_test):
    forecast = clf.predict(X_test)
    return forecast

In [8]:
def sliding_window(data,n_windows,train_size):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }

    tam = len(data)
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            scaler = StandardScaler()

            
#             Xtrain = ttrain.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
#             ytrain = ttrain.loc[:,'Appliances(t)':'Tdewpoint(t)']
#             Xtest = ttest.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
#             ytest = ttest.loc[:,'Appliances(t)':'Tdewpoint(t)']
            
            Xtrain = scaler.fit_transform(ttrain.loc[:,'CO(GT)(t-1)':'AH(t-1)'])
            ytrain = scaler.fit_transform(ttrain.loc[:,'CO(GT)(t)':'AH(t)'])
            Xtest = scaler.transform(ttest.loc[:,'CO(GT)(t-1)':'AH(t-1)'])
            ytest = scaler.transform(ttest.loc[:,'CO(GT)(t)':'AH(t)'])
            
            
            model = create_and_fit_model(Xtrain,ytrain)
            forecast = forecast_model(model, Xtest)
             
            df_forecats_columns = ttest.loc[:,'CO(GT)(t)':'AH(t)'].columns
            
            columns = list(df_forecats_columns)
            
            prediction = scaler.inverse_transform(forecast) 
            #prediction = forecast
            df_forecast = pd.DataFrame(prediction,columns=columns)
            
            #ytest_metric = scaler.inverse_transform(y_test.reshape(y_test.shape[0], y_test.shape[2]))
            ytest_metric = ttest.loc[:,'CO(GT)(t)':'AH(t)'].values
            df_original = pd.DataFrame(ytest_metric,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    return measures

In [9]:
n_windows = 30
train_size = 0.75 

model_rf = sliding_window(data,n_windows,train_size)

--------------------
training window 0
--------------------
training window 311
--------------------
training window 622
--------------------
training window 933
--------------------
training window 1244


<ipython-input-3-1be783193a86>:3: RuntimeWarning: invalid value encountered in double_scalars
  return (rmse/x)


--------------------
training window 1555


<ipython-input-3-1be783193a86>:3: RuntimeWarning: invalid value encountered in double_scalars
  return (rmse/x)


--------------------
training window 1866


...

<ipython-input-3-1be783193a86>:3: RuntimeWarning: invalid value encountered in double_scalars
  return (rmse/x)


--------------------
training window 8708


<ipython-input-3-1be783193a86>:3: RuntimeWarning: invalid value encountered in double_scalars
  return (rmse/x)


--------------------
training window 9019


<ipython-input-3-1be783193a86>:3: RuntimeWarning: invalid value encountered in double_scalars
  return (rmse/x)


In [10]:
df_forecats_columns = data.loc[:,'CO(GT)(t)':'AH(t)'].columns

columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

model_rf.to_csv (r'all_windows_random_forest_uci_air_quality_italy.csv', index = False, header=True)

measures = model_rf
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    
    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures = round(pd.DataFrame(final_result),3)

In [11]:
final_measures

,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,CO(GT)(t),30.574,14.932,955.273,0.274,39.386,inf,20.517,17.061,1740.479,0.575,25.158,NaN
1,PT08.S1(CO)(t),165.467,114.691,23.332,0.440,8.436,0.162,141.596,98.232,51.128,0.356,13.487,0.075
2,NMHC(GT)(t),25.304,17.751,7.678,0.933,3.838,NaN,72.692,56.793,26.102,0.240,13.267,NaN
3,C6H6(GT)(t),18.793,10.596,93.074,-1.149,24.787,0.247,30.197,19.701,157.139,5.633,16.708,0.276
4,PT08.S2(NMHC)(t),183.109,131.299,25.883,0.532,9.631,0.150,122.603,89.610,46.863,0.264,11.494,0.059
5,NOx(GT)(t),110.561,77.578,44.697,0.446,24.771,inf,55.363,47.079,20.814,0.251,14.638,NaN
6,PT08.S3(NOx)(t),165.105,119.188,27.890,0.504,9.739,0.149,164.627,117.215,60.331,0.498,13.212,0.096
7,NO2(GT)(t),71.733,45.307,37.534,0.206,24.249,inf,36.462,31.371,20.150,0.487,16.153,NaN
8,PT08.S4(NO2)(t),229.197,157.247,30.789,0.374,8.324,0.159,221.149,154.338,79.857,0.396,12.334,0.073
9,PT08.S5(O3)(t),218.473,164.159,27.885,0.550,11.013,0.148,101.158,72.838,37.517,0.270,9.816,0.048


In [13]:
final_measures.to_csv (r'random_forest_uci_air_quality_italy_with_std.csv', index = False, header=True)

In [53]:
# y_p=clf.predict(df_testx)
# columns=df_original.columns
# y_p=pd.DataFrame(y_p,columns=columns)
# y_p

In [54]:
# y_p.shape,df_original.shape

In [55]:

# col=df_original.columns
# result = {
        
#          "rmse": [],
         
#          "mae": [],
#          "r2": [],
#          "mape":[],
#          "variable":[]
#     }
    
# final_result = {
        
#          "rmse": [],
         
#          "mae": [],
#          "r2": [],
#          "mape":[],
#          "variable":[]}
# for col in col: 
#   original = df_original[col].values
#   forecast = y_p[col].values
#   mae = round(mean_absolute_error(original,forecast),3)
#   r2 = round(r2_score(original,forecast),3)
#   rmse = round(mean_squared_error(original,forecast,squared=False),3)             
#   mape = round(mean_absolute_percentage_error(original,forecast),3)

#   result["rmse"].append(rmse)
#   result["mae"].append(mae)
#   result["r2"].append(r2)
#   result["mape"].append(mape)
  
#   result["variable"].append(col)
        
# measures = pd.DataFrame(result)

In [57]:

# import statistics
# final_result = {
#     "variable": [],
#     "rmse": [],
#     # "nrmse": [],
#     "mae": [],
#     "r2": [],
#     "mape":[]
# }


# var = measures.groupby("variable")

# for col in columns:
    
#     var_agr = var.get_group(col)
           
#     rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
#     # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
#     mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
#     r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
#     mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

#     final_result["variable"].append(col)
#     final_result["rmse"].append(rmse)
#     # final_result["nrmse"].append(nrmse)
#     final_result["mae"].append(mae)
#     final_result["r2"].append(r2)
#     final_result["mape"].append(mape)
        
#     print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
# final_measures_kpca = pd.DataFrame(final_result) 

# print("Statistics MIMO (test): ")
# final_measures_kpca

## RandomizedSearchCV

In [6]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
# Define the hyperparameter configuration space
hyperparameters = dict(
                     estimator__n_estimators=[20, 50, 100, 200, 300],
                     estimator__criterion=['friedman_mse', 'mse'], estimator__min_samples_split=list(range(2,11)),
                     estimator__max_depth=list(range(2,30)), estimator__min_samples_leaf=list(range(1,11)),
                     estimator__max_leaf_nodes=[5, 10, 20, 30, 50])

clf=RandomForestRegressor()
model=MultiOutputRegressor(clf)
gsc = RandomizedSearchCV(model,hyperparameters,cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1,n_iter=5)

hyperparameters_tuning = gsc.fit(df_trainx, df_trainy)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning.best_estimator_

Best Parameters = {'estimator__n_estimators': 300, 'estimator__min_samples_split': 2, 'estimator__min_samples_leaf': 2, 'estimator__max_leaf_nodes': 20, 'estimator__max_depth': 21, 'estimator__criterion': 'friedman_mse'}


In [7]:
hyperparameters_tuning.best_estimator_

MultiOutputRegressor(estimator=RandomForestRegressor(criterion='friedman_mse',
                                                     max_depth=21,
                                                     max_leaf_nodes=20,
                                                     min_samples_leaf=2,
                                                     n_estimators=300))

In [8]:
y_p=tuned_model.predict(df_testx)
columns=df_original.columns
y_p=pd.DataFrame(y_p,columns=columns)
y_p

,CO(GT)(t),PT08.S1(CO)(t),NMHC(GT)(t),C6H6(GT)(t),PT08.S2(NMHC)(t),NOx(GT)(t),PT08.S3(NOx)(t),NO2(GT)(t),PT08.S4(NO2)(t),PT08.S5(O3)(t),T(t),RH(t),AH(t)
0,0.588792,1095.602218,-199.677829,8.669562,932.766063,219.852988,770.907031,113.875478,1519.758567,1009.781332,24.320047,46.145382,1.024448
1,-183.617762,912.687249,-199.677829,3.808175,731.558087,45.868274,885.587529,39.381991,1500.726558,799.802653,22.370416,52.975200,1.024448
2,0.588792,1075.449597,-199.677829,8.642669,924.959805,396.831992,707.466813,179.892251,1131.223443,1044.453543,7.454042,46.616456,0.816141
3,0.588792,1187.372964,-199.677829,7.921510,871.233277,310.370261,743.945577,117.136383,1334.235779,1011.597407,12.458665,67.757694,0.922548
4,0.588792,872.404049,-199.677829,3.487559,647.544624,105.346133,1088.317364,57.968004,917.547598,778.710942,12.221167,39.039251,0.816141
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2334,0.588792,1005.270400,-199.677829,9.447511,984.138097,359.904930,783.254089,134.557904,1237.228769,1009.478390,15.708160,39.172833,0.820187
2335,-194.796455,773.682277,-199.677829,3.478400,621.761028,-184.405799,1218.217733,-188.900726,1178.777250,619.521580,12.901422,67.703053,0.922548
2336,0.588792,-155.778188,-199.677829,-192.037522,-158.513354,994.230875,-168.867499,207.734907,-148.299519,-139.181629,-189.591929,-192.161239,-187.816505
2337,0.588792,854.734437,-199.677829,3.478400,635.302328,32.587173,1091.283645,13.621035,1355.222497,545.881584,16.558718,77.283788,1.024448


In [9]:
col=df_original.columns
result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }
    
final_result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]}
for col in col: 
  original = df_original[col].values
  forecast = y_p[col].values
  mae = round(mean_absolute_error(original,forecast),3)
  r2 = round(r2_score(original,forecast),3)
  rmse = round(mean_squared_error(original,forecast,squared=False),3)             
  mape = round(mean_absolute_percentage_error(original,forecast),3)

  result["rmse"].append(rmse)
  result["mae"].append(mae)
  result["r2"].append(r2)
  result["mape"].append(mape)
  
  result["variable"].append(col)
        
measures = pd.DataFrame(result)

In [10]:
import statistics
final_result = {
    "variable": [],
    "rmse": [],
    # "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}


var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    # final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)
        
    print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('CO(GT)(t)', 30.343, 8.999, 0.842, 6.829)
Results: ('PT08.S1(CO)(t)', 126.784, 77.849, 0.859, 0.087)
Results: ('NMHC(GT)(t)', 42.93, 9.358, 0.923, 0.054)
Results: ('C6H6(GT)(t)', 12.56, 3.85, 0.911, 0.458)
Results: ('PT08.S2(NMHC)(t)', 146.853, 98.755, 0.826, 0.12)
Results: ('NOx(GT)(t)', 110.98, 66.605, 0.817, 0.367)
Results: ('PT08.S3(NOx)(t)', 119.289, 80.22, 0.868, 0.114)
Results: ('NO2(GT)(t)', 61.058, 30.135, 0.768, 0.264)
Results: ('PT08.S4(NO2)(t)', 167.011, 100.659, 0.877, 0.09)
Results: ('PT08.S5(O3)(t)', 187.17, 131.036, 0.839, 0.155)
Results: ('T(t)', 12.363, 2.441, 0.92, 0.145)
Results: ('RH(t)', 14.655, 4.76, 0.92, 0.097)
Results: ('AH(t)', 11.303, 1.432, 0.918, 0.924)
Statistics MIMO (test): 


,variable,rmse,mae,r2,mape
0,CO(GT)(t),30.343,8.999,0.842,6.829
1,PT08.S1(CO)(t),126.784,77.849,0.859,0.087
2,NMHC(GT)(t),42.930,9.358,0.923,0.054
3,C6H6(GT)(t),12.560,3.850,0.911,0.458
4,PT08.S2(NMHC)(t),146.853,98.755,0.826,0.120
5,NOx(GT)(t),110.980,66.605,0.817,0.367
6,PT08.S3(NOx)(t),119.289,80.220,0.868,0.114
7,NO2(GT)(t),61.058,30.135,0.768,0.264
8,PT08.S4(NO2)(t),167.011,100.659,0.877,0.090
9,PT08.S5(O3)(t),187.170,131.036,0.839,0.155


## MinMaxScaler

In [11]:
x,y=data.loc[:,:],data.loc[:,:]

In [12]:
from sklearn.preprocessing import MinMaxScaler
scalerx = MinMaxScaler()
scalery=MinMaxScaler()

In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# Get the number of inputs and outputs from the dataset
df_trainx=x_train.loc[:,'CO(GT)(t-1)':'AH(t-1)']
df_trainx_scaled = scalerx.fit_transform(df_trainx)
df_testx=x_test.loc[:,'CO(GT)(t-1)':'AH(t-1)']
df_testx_scaled = scalerx.transform(df_testx)
df_trainy=y_train.loc[:,'CO(GT)(t)':'AH(t)']
df_trainy_scaled=scalery.fit_transform(df_trainy)

df_original=y_test.loc[:,'CO(GT)(t)':'AH(t)']

n_inputs, n_outputs = df_trainx.shape[1], df_trainy.shape[1]

In [14]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
# Define the hyperparameter configuration space
hyperparameters = dict(
                     estimator__n_estimators=[20, 50, 100, 200, 300],
                     estimator__min_samples_split=list(range(2,11)),
                     estimator__max_depth=[2,5,10,15,20,30], estimator__min_samples_leaf=list(range(1,11)),
                     estimator__max_leaf_nodes=[5, 10, 20, 30, 50])

clf=RandomForestRegressor()
model=MultiOutputRegressor(clf)
gsc = RandomizedSearchCV(model,hyperparameters,cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1,n_iter=5)

hyperparameters_tuning = gsc.fit(df_trainx_scaled, df_trainy_scaled)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning.best_estimator_

Best Parameters = {'estimator__n_estimators': 50, 'estimator__min_samples_split': 2, 'estimator__min_samples_leaf': 6, 'estimator__max_leaf_nodes': 50, 'estimator__max_depth': 20}


In [15]:
hyperparameters_tuning.best_estimator_

MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=20,
                                                     max_leaf_nodes=50,
                                                     min_samples_leaf=6,
                                                     n_estimators=50))

In [16]:
y_p=tuned_model.predict(df_testx_scaled)
columns=df_original.columns
y_p=pd.DataFrame(y_p,columns=columns)
y_p

,CO(GT)(t),PT08.S1(CO)(t),NMHC(GT)(t),C6H6(GT)(t),PT08.S2(NMHC)(t),NOx(GT)(t),PT08.S3(NOx)(t),NO2(GT)(t),PT08.S4(NO2)(t),PT08.S5(O3)(t),T(t),RH(t),AH(t)
0,0.958715,0.577593,0.000097,0.793253,0.468896,0.251051,0.338282,0.579732,0.577878,0.437590,0.914480,0.857390,0.996234
1,0.071249,0.492160,0.000097,0.776010,0.390397,0.146710,0.372713,0.395604,0.576562,0.363876,0.911656,0.880382,0.996365
2,0.960186,0.562464,0.000097,0.792380,0.458444,0.337483,0.314846,0.709881,0.446678,0.453117,0.846466,0.863993,0.991659
3,0.959263,0.630568,0.000097,0.786431,0.428186,0.301266,0.327245,0.584748,0.513664,0.447120,0.872614,0.935888,0.994468
4,0.951400,0.479474,0.000097,0.769482,0.353994,0.180004,0.448517,0.490733,0.375100,0.356515,0.861901,0.831481,0.991643
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2334,0.960193,0.537087,0.000097,0.797319,0.476048,0.324513,0.343288,0.627969,0.477436,0.441205,0.878113,0.837744,0.992465
2335,0.006093,0.433233,0.000097,0.766436,0.333179,0.007477,0.489208,0.019392,0.463408,0.297393,0.873555,0.930241,0.994423
2336,0.963720,0.019045,0.000097,0.009169,0.009738,0.740879,0.007986,0.761745,0.009161,0.016471,0.005683,0.013388,0.016691
2337,0.929985,0.469734,0.000097,0.766794,0.337323,0.138014,0.441263,0.394659,0.520039,0.276748,0.889403,0.961858,0.996958


In [17]:
y_p_inv = scalery.inverse_transform(y_p)
columns=df_original.columns
y_p_inv=pd.DataFrame(y_p_inv,columns=columns)
y_p_inv

,CO(GT)(t),PT08.S1(CO)(t),NMHC(GT)(t),C6H6(GT)(t),PT08.S2(NMHC)(t),NOx(GT)(t),PT08.S3(NOx)(t),NO2(GT)(t),PT08.S4(NO2)(t),PT08.S5(O3)(t),T(t),RH(t),AH(t)
0,1.521957,1093.807604,-199.865806,9.180884,931.915340,221.514744,775.265775,113.055442,1519.187180,991.556479,23.681698,46.156682,1.415272
1,-185.023447,902.437514,-199.865806,4.633891,742.418650,46.326249,874.532701,13.626255,1515.270718,790.833473,22.991016,52.757695,1.441614
2,1.831005,1059.920423,-199.865806,8.950512,906.683570,366.634699,707.700659,183.335936,1128.866759,1033.837524,7.045487,48.052257,0.490263
3,1.637161,1212.472406,-199.865806,7.381865,833.641513,305.825552,743.446014,115.763674,1328.151435,1017.506620,13.441419,68.693446,1.058131
4,-0.015645,874.022630,-199.865806,2.912384,654.540506,102.226693,1093.074649,64.996008,915.922937,770.790654,10.820908,38.718229,0.486939
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2334,1.832615,1003.074260,-199.865806,10.252978,949.179278,344.857336,789.698620,139.103442,1220.371716,1001.400274,14.786385,40.516336,0.653131
2335,-198.719211,770.442757,-199.865806,2.109193,604.293196,-187.446551,1210.387233,-189.528127,1178.637469,609.801513,13.671546,67.072310,1.049076
2336,2.573956,-157.340218,-199.865806,-197.582250,-176.492371,1043.935610,-176.975136,211.342458,-172.747290,-155.149931,-198.610000,-196.156407,-196.625391
2337,-4.517133,852.203525,-199.865806,2.203673,614.297223,31.725674,1072.161873,13.115713,1347.115932,553.585900,17.547975,76.149544,1.561673


In [18]:
df_original

,CO(GT)(t),PT08.S1(CO)(t),NMHC(GT)(t),C6H6(GT)(t),PT08.S2(NMHC)(t),NOx(GT)(t),PT08.S3(NOx)(t),NO2(GT)(t),PT08.S4(NO2)(t),PT08.S5(O3)(t),T(t),RH(t),AH(t)
4636,1.7,1077.0,-200.0,8.5,920.0,169.0,778.0,99.0,1509.0,935.0,23.6,50.0,1.4383
3396,-200.0,894.0,-200.0,4.9,761.0,36.0,862.0,48.0,1504.0,768.0,23.0,49.9,1.3849
8632,1.1,915.0,-200.0,3.1,658.0,166.0,957.0,121.0,866.0,562.0,10.0,37.7,0.4643
6984,2.3,1286.0,-200.0,9.1,942.0,343.0,687.0,137.0,1362.0,1103.0,13.5,67.1,1.0357
6612,1.0,909.0,-200.0,4.3,728.0,221.0,933.0,107.0,948.0,905.0,7.8,47.4,0.5046
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5810,3.1,1064.0,-200.0,12.2,1055.0,412.0,738.0,138.0,1242.0,1063.0,14.1,43.0,0.6883
5314,-200.0,777.0,-200.0,1.8,570.0,-200.0,1191.0,-200.0,1167.0,616.0,14.5,65.0,1.0672
6747,8.6,-200.0,-200.0,-200.0,-200.0,1237.0,-200.0,203.0,-200.0,-200.0,-200.0,-200.0,-200.0000
5147,0.6,903.0,-200.0,3.4,679.0,90.0,956.0,44.0,1385.0,660.0,18.2,76.9,1.5945


In [19]:
col=df_original.columns
result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }
    
final_result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]}
for col in col: 
  original = df_original[col].values
  forecast = y_p_inv[col].values
  mae = round(mean_absolute_error(original,forecast),3)
  r2 = round(r2_score(original,forecast),3)
  rmse = round(mean_squared_error(original,forecast,squared=False),3)             
  mape = round(mean_absolute_percentage_error(original,forecast),3)

  result["rmse"].append(rmse)
  result["mae"].append(mae)
  result["r2"].append(r2)
  result["mape"].append(mape)
  
  result["variable"].append(col)
        
measures = pd.DataFrame(result)

In [20]:
import statistics
final_result = {
    "variable": [],
    "rmse": [],
    # "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}


var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    # final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)
        
    print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('CO(GT)(t)', 30.413, 8.954, 0.841, 8.044)
Results: ('PT08.S1(CO)(t)', 124.131, 76.148, 0.865, 0.086)
Results: ('NMHC(GT)(t)', 43.095, 9.321, 0.922, 0.049)
Results: ('C6H6(GT)(t)', 12.32, 3.735, 0.914, 0.384)
Results: ('PT08.S2(NMHC)(t)', 143.857, 96.811, 0.833, 0.118)
Results: ('NOx(GT)(t)', 109.61, 65.412, 0.822, 0.358)
Results: ('PT08.S3(NOx)(t)', 117.901, 79.189, 0.871, 0.112)
Results: ('NO2(GT)(t)', 60.805, 30.436, 0.769, 0.271)
Results: ('PT08.S4(NO2)(t)', 164.01, 99.134, 0.881, 0.089)
Results: ('PT08.S5(O3)(t)', 184.196, 128.546, 0.844, 0.153)
Results: ('T(t)', 12.169, 2.395, 0.923, 0.125)
Results: ('RH(t)', 14.483, 4.616, 0.922, 0.093)
Results: ('AH(t)', 11.161, 1.346, 0.92, 0.772)
Statistics MIMO (test): 


,variable,rmse,mae,r2,mape
0,CO(GT)(t),30.413,8.954,0.841,8.044
1,PT08.S1(CO)(t),124.131,76.148,0.865,0.086
2,NMHC(GT)(t),43.095,9.321,0.922,0.049
3,C6H6(GT)(t),12.320,3.735,0.914,0.384
4,PT08.S2(NMHC)(t),143.857,96.811,0.833,0.118
5,NOx(GT)(t),109.610,65.412,0.822,0.358
6,PT08.S3(NOx)(t),117.901,79.189,0.871,0.112
7,NO2(GT)(t),60.805,30.436,0.769,0.271
8,PT08.S4(NO2)(t),164.010,99.134,0.881,0.089
9,PT08.S5(O3)(t),184.196,128.546,0.844,0.153
